In [2]:
import gradio as gr
from gradio import ChatMessage
from pathlib import Path

from config import CONFIG
from skimmer import PDFSkimmer
from chatbot import Chatbot
from db_manager import DatabaseManager

BASE_DIR = CONFIG["directories"]["base_dir"]
BASE_URL = CONFIG["directories"]["base_url"]


/Volumes/Aquatope/_DEV_/PDF_Skimmer_v2/.venv/lib/python3.13/site-packages/langchain_deepseek/__init__.py:2: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from .base import ChatDeepSeekAI


In [2]:
skimmer = PDFSkimmer()
chatbot = Chatbot(type="analysis")


def get_summary(file_path: str):
    return skimmer.load_or_summary(file_path)


In [3]:
file_path = "/Volumes/AnselmJeong.ddns.net/LIBRARY/Articles/d41586-025-00068-5.pdf"

In [4]:
response = get_summary(file_path)
print(response)

No summary found for /Volumes/AnselmJeong.ddns.net/LIBRARY/Articles/d41586-025-00068-5.pdf
URL: http://anselmjeong.synology.me:8091/share/GXzzOcfK/d41586-025-00068-5.pdf
Response from the LLM:

{'core_question': 'What is the core research question?', 'introduction': 'What is the background and context of the study?', 'methodology': 'What is the methodology of the study?', 'results': 'What are the key results and findings?', 'discussion': 'What are the implications of the results?', 'limitations': 'What are the limitations of the study?'}


TypeError: custom_types.DatabaseSummary() argument after ** must be a mapping, not PaperSummary

In [ ]:
from article_analysis import ArticleAnalyzer

AA = ArticleAnalyzer()
response = AA.summarize(file_path)


In [5]:
chatbot.set_url("http://anselmjeong.synology.me:8091/share/GXzzOcfK/d41586-025-00068-5.pdf")

In [ ]:
chatbot.query_llm()

In [ ]:
chatbot.history_langchain

In [ ]:
def generate_chatinterface(file_path: str):
    def _handle_chat(input: str, history: list):
        history.append({"role": "user", "content": input})

        # response = chatbot.query_llm(input, history)

        response = "This is a test response"
        history.append({"role": "assistant", "content": response})
        return "", history

    gr.Markdown("# Chat with a LangChain Agent")
    chatbot = gr.Chatbot(
        type="messages",
        label="Langchain Agent",
    )
    input = gr.Textbox(lines=1, label="Chat Message")
    input.submit(_handle_chat, [input, chatbot], [input, chatbot])


In [2]:
db_manager = DatabaseManager()

with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            file_explorer = gr.FileExplorer(glob="**/*.pdf", root_dir=BASE_DIR, file_count="single")
        with gr.Column():
            gr.Markdown("# Chat with a LangChain Agent")
            chatbox = gr.Chatbot(
                type="messages",
                label="Langchain Agent",
            )
            with gr.Group():
                with gr.Row():
                    input = gr.Textbox(lines=1, scale=10, show_label=False, container=False)
                    save_button = gr.Button("Save", scale=1)

            def load_chat_history(file_path: str):
                try:
                    file_name = Path(file_path).name
                    chat_record = db_manager.get_chat_history(file_name)
                    if chat_record:
                        return chat_record
                    return []
                except Exception as e:
                    print(f"Error loading chat history: {str(e)}")
                    return []

            # Add file_explorer change event to load chat history
            file_explorer.change(fn=load_chat_history, inputs=[file_explorer], outputs=[chatbox])

            def save_chat_history(file_path: str, history: list):
                if not file_path:
                    return gr.Error("Please select a file first")
                try:
                    file_name = Path(file_path).name
                    db_manager.save_chat_history(file_name, history)
                    gr.Info("Chat history saved successfully")
                except Exception as e:
                    gr.Error(f"Error saving chat history: {str(e)}")

            def _handle_chat(input: str, history: list):
                history.append({"role": "user", "content": input})

                # response = chatbot.query_llm(input, history)

                response = f"This is a {len(history)} test response"
                history.append({"role": "assistant", "content": response})
                return "", history

            save_button.click(fn=save_chat_history, inputs=[file_explorer, chatbox])

            input.submit(_handle_chat, [input, chatbox], [input, chatbox])

    # file_explorer.change(fn=get_summary, inputs=file_explorer, outputs=[result])


demo.launch()


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [3]:
from pathlib import Path
import urllib.parse


def get_file_url(file_path: str | Path, base_url: str = BASE_URL) -> str:
    if isinstance(file_path, str):
        file_path = Path(file_path)
    # Remove the BASE_DIR part from the file_path
    if str(file_path).startswith(BASE_DIR):
        file_path = file_path.relative_to(BASE_DIR)
    # URL encode the filename
    encoded_filename = urllib.parse.quote(str(file_path))

    # Combine base URL with encoded filename
    full_url = "/".join([base_url, encoded_filename])

    return str(full_url)


In [4]:
from config import CONFIG

BASE_DIR = CONFIG["directories"]["base_dir"]
BASE_URL = CONFIG["directories"]["base_url"]
file_path = "/Volumes/LIBRARY/Articles/Artificial Intelligence and Machine Learning/2017 - Orhan et al. - Efficient probabilistic inference in generic neural networks trained with non-probabilistic feedback.pdf"
url = get_file_url(file_path, base_url=BASE_URL)
print(url)


http://anselmjeong.synology.me:8091/share/GXzzOcfK/Artificial%20Intelligence%20and%20Machine%20Learning/2017%20-%20Orhan%20et%20al.%20-%20Efficient%20probabilistic%20inference%20in%20generic%20neural%20networks%20trained%20with%20non-probabilistic%20feedback.pdf
